In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from scipy.io import loadmat
import cv2

def load_hoda(training_sample_size=3500, test_sample_size=400, size=28):
    dataset = loadmat('dataset/Data_hoda_full.mat')
    X_train_original = np.squeeze(dataset['Data'][:training_sample_size])
    y_train = np.squeeze(dataset['labels'][:training_sample_size])
    X_test_original = np.squeeze(dataset['Data'][training_sample_size:training_sample_size+test_sample_size])
    y_test = np.squeeze(dataset['labels'][training_sample_size:training_sample_size+test_sample_size])

    X_train_resized = [cv2.resize(img, (size, size)) for img in X_train_original]
    X_test_resized = [cv2.resize(img, (size, size)) for img in X_test_original]

    # reshape برای CNN
    X_train = np.array(X_train_resized).reshape(-1, size, size, 1)
    X_test = np.array(X_test_resized).reshape(-1, size, size, 1)

    return X_train, y_train, X_test, y_test


In [ ]:
x_train, y_train, x_test, y_test = load_hoda(training_sample_size=3500, test_sample_size=400, size=28)

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# تبدیل لیبل ها به One-hot
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

x_val = x_test[:200]
y_val = y_test[:200]
x_test = x_test[200:]
y_test = y_test[200:]


In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])


In [ ]:
class StopAtAccuracy(tf.keras.callbacks.Callback):
    def __init__(self, target=0.75):
        super().__init__()
        self.target = target

    def on_epoch_end(self, epoch, logs=None):
        val_acc = logs.get("val_categorical_accuracy")
        if val_acc is not None and val_acc >= self.target:
            print(f"\nدقت اعتبارسنجی به {self.target*100:.1f}% رسید، آموزش متوقف شد.")
            self.model.stop_training = True

stop_callback = StopAtAccuracy(target=0.75)


In [ ]:
history = model.fit(
    x_train, y_train,
    epochs=30,
    batch_size=256,
    validation_data=(x_val, y_val),
    callbacks=[stop_callback],
    verbose=1
)


Epoch 1/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 78ms/step - categorical_accuracy: 0.3037 - loss: 2.0639 - val_categorical_accuracy: 0.7450 - val_loss: 1.3877
Epoch 2/30
13/14 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - categorical_accuracy: 0.5051 - loss: 1.4187
دقت اعتبارسنجی به 75.0% رسید، آموزش متوقف شد.
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - categorical_accuracy: 0.5683 - loss: 1.2487 - val_categorical_accuracy: 0.8200 - val_loss: 0.5869
